# Imports des json

In [1]:
bounces_path = "../outputs/bounces.json"
hits_path = "../outputs/hits.json"
players_path = "../outputs/players.json"
trajectories_path = "../outputs/trajectories.json"
frames_path = "../data/frames_data.json"

In [2]:
import json

with open(bounces_path, 'r', encoding='utf-8') as f1:
    bounces_data = json.load(f1)

with open(hits_path, 'r', encoding='utf-8') as f2:
    hits_data = json.load(f2)

with open(players_path, 'r', encoding='utf-8') as f3:
    players_data = json.load(f3)

with open(trajectories_path, 'r', encoding='utf-8') as f4:
    trajectories_data = json.load(f4)

with open(frames_path, 'r', encoding='utf-8') as f5:
    frames_data = json.load(f5) 

# Création de la minimap

In [3]:
from minimap import generate_minimap_video

In [4]:
video_path_input = "../data/videos/2.mp4"
video_path_output = "../outputs/videos/2_boug.mp4"

In [9]:
generate_minimap_video(video_path_input, video_path_output, frames_data, bounces_data=bounces_data, hits_data=hits_data, players_data=players_data, trajectories_data=trajectories_data, plot_players_tail=False, width_goal=500)

# Création vidéo finale

In [11]:
video_path_graph = "../outputs/videos/players_speed_analysis.mp4"
video_path_table = "../outputs/videos/table_stats.mp4"

In [17]:
import cv2
import numpy as np
from pathlib import Path

def assemble_four_videos(
    video1_path: str,                 # Vidéo de base (seront redim. à 1920×1080)
    video2_path: str,                 # Clip à droite (redim. à 500×1080)
    video3_path: str,                 # Clip sous la base, largeur variable
    video4_path: str,                 # Clip sous vidéo2, largeur ajustée
    output_path: str,
    bottom_h: int = 900,              # Hauteur commune vidéos 3 & 4
    bottom_w_left: int = 1210,        # Largeur vidéo 3
    fourcc: str = "avc1",             # "avc1" = H.264 ; sinon "mp4v"
):
    """
    Crée une mosaïque :
          [ Vidéo1 1920×1080 ] [ Vidéo2 500×1080 ]
          [ Vidéo3 bottom_w_left×bottom_h ] [ Vidéo4 auto×bottom_h ]

    • Vidéo1 et Vidéo2 conservent les dimensions fixes (1920×1080, 500×1080).
    • Vidéo3 et Vidéo4 conservent `bottom_h` ; Vidéo3 garde width=bottom_w_left,
      Vidéo4 prend le reste pour totaliser 2420 px de large.
    • Pas d’audio (OpenCV ne gère pas l’audio).
    """

    # Dimensions fixes du haut
    TOP_H   = 1080
    W1, W2  = 1920, 500
    TOTAL_W = W1 + W2  # 2420 px

    # Contrôles
    if bottom_w_left <= 0 or bottom_w_left >= TOTAL_W:
        raise ValueError(f"bottom_w_left doit être entre 1 et {TOTAL_W-1}")
    if bottom_h <= 0:
        raise ValueError("bottom_h doit être > 0")

    bottom_w_right = TOTAL_W - bottom_w_left
    TOTAL_H        = TOP_H + bottom_h

    # 1. Ouverture des 4 vidéos ------------------------------------------------
    caps = [cv2.VideoCapture(p) for p in (video1_path, video2_path, video3_path, video4_path)]
    if not all(c.isOpened() for c in caps):
        raise FileNotFoundError("Au moins une vidéo ne s’ouvre pas.")

    # FPS : on prend celui de la vidéo1
    fps = caps[0].get(cv2.CAP_PROP_FPS) or 30
    frame_counts = [int(c.get(cv2.CAP_PROP_FRAME_COUNT)) for c in caps]
    common_frames = min(frame_counts)          # on coupe au plus court

    # 2. Préparation du VideoWriter -------------------------------------------
    writer = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter_fourcc(*fourcc),
        fps,
        (TOTAL_W, TOTAL_H),
        isColor=True,
    )

    # 3. Boucle d’assemblage ---------------------------------------------------
    for _ in range(common_frames):
        rets_frames = [cap.read() for cap in caps]
        if not all(ret for ret, _ in rets_frames):
            break  # sécurité

        f1, f2, f3, f4 = (fr for _, fr in rets_frames)

        # Redimensionnements
        f1 = cv2.resize(f1, (W1, TOP_H))
        f2 = cv2.resize(f2, (W2, TOP_H))
        f3 = cv2.resize(f3, (bottom_w_left,  bottom_h))
        f4 = cv2.resize(f4, (bottom_w_right, bottom_h))

        # Toile noire
        canvas = np.zeros((TOTAL_H, TOTAL_W, 3), dtype=np.uint8)

        # Placement (y, x)
        canvas[0:TOP_H,      0:W1]               = f1              # Vidéo 1
        canvas[0:TOP_H,      W1:TOTAL_W]         = f2              # Vidéo 2
        canvas[TOP_H:TOTAL_H, 0:bottom_w_left]   = f3              # Vidéo 3
        canvas[TOP_H:TOTAL_H, bottom_w_left:TOTAL_W] = f4          # Vidéo 4

        writer.write(canvas)

    # 4. Nettoyage -------------------------------------------------------------
    for cap in caps:
        cap.release()
    writer.release()
    print(f"✅ Vidéo écrite : {Path(output_path).resolve()}")

# ---------------------------------------------------------------------------
# Exemple d’usage -----------------------------------------------------------
assemble_four_videos(
    video1_path = video_path_input,      # 1920×1080
    video2_path = video_path_output,     # 500×1080
    video3_path = video_path_graph,      # largeur param.
    video4_path = video_path_table,      # auto
    output_path = "../outputs/videos/boug_final.mp4",
    bottom_h       = 500,    # hauteur Vidéos 3 & 4
    bottom_w_left  = 1800,   # largeur Vidéo 3
)


✅ Vidéo écrite : /Users/elouann/Desktop/Elouann/Études/Polytechnique/Professionnel/Stages/Stage 3A/Quantum Analytics Sport/Boulot/Tennis/tennis-ai/outputs/videos/boug_final.mp4
